# Data Analysis and Feature Selection

 **Defined Code Functions** 

In [1]:
# Code objective:(1) Import subject airbag data and .xls data sheet with fall event indices to segment data, 
# (2) determine the impact time of the fall within the segment, (3) create a 1s wide window 75ms before impact,
# (4) calculate features within the window space, #(5) export features onto .xls sheet and re-loop for each fall,
# (6) determine best features for multiple types ML based on trial and error.

**Libraries**

In [29]:
# Open library functions
import xlrd #to open and import excel data
from pathlib import Path #to concatenate file name
import pandas as pd #to perforn dataframe calulcations
import numpy as np #to perform mathematical equations
import scipy.io as sio #to load matrix data from .mat file
import math
import sklearn # to process data, access ML algorithms

# ML Required Libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
# SVM
from sklearn.svm import SVC 
# KNN
from sklearn.neighbors import KNeighborsClassifier

**Import Data**

In [3]:
# Get subject information
subject = input('Subject ID: ')
dataFolder = Path("Y:/Fall Catcher/Exp Data/")
subjectFolder = dataFolder/subject

# Import .xls combined data sheet per subject
eventFile = subject + '_CombinedData.xls'
eventDataLoc = subjectFolder/eventFile
eventdf = pd.read_excel(eventDataLoc)

# Import .mat airbag data file per subject
airbagFolder = subjectFolder/'.MAT files'
airbagWS = airbagFolder/'Airbag.mat'
Airbag = sio.loadmat(airbagWS)
airbagMat = Airbag.get('Airbag','Entry not found.')
airbagHeaders = ['t','bAx','bAy','bAz','bGx','bGy','bGz','rAx','rAy','rAz','rGx','rGy','rGz','lAx','lAy','lAz','lGx','lGy','lGz','output']
airbagdf = pd.DataFrame(airbagMat,columns = airbagHeaders)

Subject ID: P2


In [6]:
# Raw data import of airbag mat file and combined subject data sheet
print(eventdf)
print(airbagdf)


    eventLabel  eventStart_UTC  impactTime_UTC   eventEnd_UTC  eventStart_ms  \
0           10   1574785637303               0  1574785645914          70833   
1           10   1574785654435               0  1574785663295          87965   
2          117   1574785825017   1574785825475  1574785828531         258547   
3          117   1574785854019   1574785854389  1574785859002         287549   
4          117   1574785880490   1574785880341  1574785884792         314020   
5          118   1574785922536   1574785922571  1574785926380         356066   
6          118   1574785944570   1574785944551  1574785949442         378100   
7          118   1574785967997   1574785968351  1574785972600         401527   
8          119   1574786009461   1574786009521  1574786012411         442991   
9          119   1574786033824   1574786034321  1574786038388         467354   
10         120   1574786080680   1574786080806  1574786085236         514210   
11         120   1574786106181   1574786

**Pre-Processing Data**

In [7]:
from sklearn import preprocessing

cols_normalized = ['bAx','bAy','bAz','bGx','bGy','bGz','rAx','rAy','rAz','rGx','rGy','rGz','lAx','lAy','lAz','lGx','lGy','lGz']
# Turn to units of G
airbagdf[cols_normalized] = airbagdf[cols_normalized].apply(lambda x: (x/(-2048)))
# Normalize airbag values between 0 and 1
airbagdf[cols_normalized] = airbagdf[cols_normalized].apply(lambda x: (x - x.min())/(x.max()-x.min()))

In [8]:
# updated airbag file after processing
print(airbagdf)


               t       bAx       bAy       bAz       bGx       bGy       bGz  \
0            512  0.437657  0.423818  0.610636  0.725239  0.263795  0.299878   
1            514  0.437567  0.423917  0.610599  0.725178  0.264029  0.299633   
2            516  0.437590  0.423818  0.610451  0.725269  0.263912  0.299511   
3            518  0.437882  0.423843  0.610525  0.725178  0.263766  0.300000   
4            520  0.437815  0.423818  0.610618  0.725116  0.263853  0.299878   
...          ...       ...       ...       ...       ...       ...       ...   
1277344  3020148  0.436848  0.425236  0.601572  0.727595  0.264293  0.298350   
1277345  3020150  0.436983  0.425187  0.601590  0.727687  0.264264  0.298533   
1277346  3020152  0.436735  0.425386  0.601127  0.727626  0.264234  0.298289   
1277347  3020154  0.436825  0.425311  0.601368  0.727595  0.264264  0.298472   
1277348  3020156  0.436870  0.425187  0.601702  0.727626  0.264205  0.298044   

              rAx       rAy       rAz  

**Identify Theoretical Impact Times and Create Fall Segment Files**

In [9]:
# For each fall, determine the true time of the impact. 
theorImpactTime = []
impactError = []

# Based on impact time, create segmented data file and append to appropriate list.
allFall_clipList = []
sideFall_clipList = []
NF_clipList = [] #non-fall.
fbFall_clipList = [] #forward or backward fall.

# Clip size in frames.
windowSize = 2000
halfWin = windowSize/2

for event in range(eventdf.shape[0]):
    # For each loop, initialize the fall data frame to be empty.
    falldf = pd.DataFrame()
    
    # Get start and stop times.
    if eventdf.eventLabel[event] == 500: # No range times documented- use time window between documented falls.
        start_range = eventdf.eventEnd_ms[event-1]
        end_range = eventdf.eventStart_ms[event+1]
    else:
        start_range = eventdf.eventStart_ms[event]
        end_range = eventdf.eventEnd_ms[event]
    
    # Get the index of the airbag time closely aligning to the start and stop marked timepoints.
    airbagStartIndex = airbagdf[airbagdf['t']>=start_range].index.values.astype(int)[0]
    airbagEndIndex =airbagdf[airbagdf['t']>= end_range].index.values.astype(int)[0]
    
    # Create a sub-data frame for the fall data.
    falldf = airbagdf.loc[airbagStartIndex:airbagEndIndex]
    falldf.columns = airbagHeaders

    
    # ATTEMPTS TO FIND TRUE / THEORETICAL IMPACT TIMES:
    # Attempt 1: Calculate difference with previous row to get largest slope change during task event.
    
    # ...
    
    # Attempt 2:  Calculate magnitude of acceleration for each sensor
    # Subset dataframe to find magnitude of acceleration for each sensor.
    bAdf = falldf[['bAx','bAy','bAz']]
    rAdf = falldf[['rAx','rAy','rAz']]
    lAdf = falldf[['lAx','lAy','lAz']]
    mag_bAdf = []
    mag_rAdf = []
    mag_lAdf = []
    
    # Get magnitude vector for each sensor.
    mag_bAdf= bAdf.apply(np.linalg.norm, axis = 1)
    mag_rAdf= rAdf.apply(np.linalg.norm, axis = 1)
    mag_lAdf= lAdf. apply(np.linalg.norm, axis = 1)

    # Get the maximum magnitude for each sensor data frame.
    maxA = []
    maxA.append(mag_bAdf.max())
    maxA.append(mag_rAdf.max())
    maxA.append(mag_lAdf.max())

    # Overall maximum magnitude value.
    valMaxA = max(maxA)
    sensorMaxA = maxA.index(max(maxA))
    if sensorMaxA == 0:
        sensorType = "base"
        indexMaxA = mag_bAdf[mag_bAdf == valMaxA].index.values.astype(int)[0]
    elif sensorMaxA == 1:
        sensorType = "right"
        indexMaxA = mag_rAdf[mag_rAdf == valMaxA].index.values.astype(int)[0]
    elif sensorMaxA == 2:
        sensorType = "left"
        indexMaxA = mag_lAdf[mag_lAdf == valMaxA].index.values.astype(int)[0]

    timeMaxA = airbagdf.t[indexMaxA]
    theorImpactTime.append(timeMaxA)
    #print('The maximum acceleration value for the fall event was',valMaxA,' given by the',sensorType,'sensor at time t = ',timeMaxA, ' ms')

    # Calculate percent error for the theoretical impact time if experimental impact time marked during session. 
    if eventdf.impactTime_ms[event] != 0:
        expImpactTime = eventdf.impactTime_ms[event]
        perError = ((abs(expImpactTime-timeMaxA))/timeMaxA)*100
        impactError.append(perError)
    else:
        perError = float("NaN")
        impactError.append(perError)
    
    # Attempt #N.....
    
    # Compare and select the method with the lowest error when compared to the manual impact time point. 
    # (if a manual time point exists)

    
    
    
    # Create the Segmented Clip for the Fall
    # Get index of airbag time for the pre-selected window size.
    event_index = airbagdf[airbagdf['t']>= timeMaxA].index.values.astype(int)[0]
    startInd = event_index - halfWin
    endInd = event_index + halfWin
    clipdf = airbagdf.loc[startInd:endInd]
    clipdf.columns = airbagHeaders
    
    # Append to fall/NF list.
    if eventdf.actualFallType[event]==2:
        sideFall_clipList.append(clipdf)
        allFall_clipList.append(clipdf)
    elif eventdf.actualFallType[event]==1:
        allFall_clipList.append(clipdf)
        fbFall_clipList.append(clipdf)
    else:
        NF_clipList.append(clipdf)        
            

In [10]:
print(theorImpactTime)

[78920, 94158, 259154, 288322, 314642, 356866, 378474, 402012, 443312, 468054, 514630, 540272, 962112, 987894, 1041712, 1068318, 1141094, 1166174, 1215068, 1241708, 1290310, 1304542, 1429190, 1633272, 1655724, 1679278, 1753328, 1800716, 1891000, 1921604, 1959136, 2444956, 2454150, 2460950, 2476615, 2484628, 2490362, 2571622, 2610464, 2643766, 2653368, 2688140, 2779452, 2788708, 2822028, 2827982, 2838152, 2843330, 2848344, 2855478, 2899486, 2902544, 2905444, 2934686, 2949630, 2959382, 2968526, 2991736, 3017148]


**Define Feature Calculations**

In [11]:
# Define feature calculations here if necessary... 


**Feature Calculations on Pre-Impact Window**

In [12]:
# Create empty dictionary for calculated features to be appended to. Fall Label (Key) : Feature Vector (Value)
fallFeaturesDict = dict()

# Set parameters for window size.
leadTime = 75
windowSize = 1000
# Set parameters for feature selection.
chosenVars = ['bAx','bAy','bAz','rAx','rAy','rAz','lAx','lAy','lAz']
calculations = ['mean()','max()','min()','std()']


for impact in range(len(theorImpactTime)):
    # Create window.
    endWindow = theorImpactTime[impact] - leadTime 
    startWindow = endWindow - windowSize
    
    # Assign airbag data index for window size.
    airbagStartWin = airbagdf[airbagdf['t']>=startWindow].index.values.astype(int)[0]
    airbagEndWin =airbagdf[airbagdf['t']>= endWindow].index.values.astype(int)[0]
    
    # Create sub data frame of airbag data within the window.
    dfWindow = airbagdf[chosenVars].loc[airbagStartWin:airbagEndWin]

    
    
    
    # PERFORM CALCULATIONS ON WINDOW OF TIME
    # Pre-allocate temporary dataframe for the window of feature data calculated.
    featureCol = [] 
    # Get and assign proper column names based on variables and functions chosen.
    for var in chosenVars:
        for func in calculations:
            featureCol.append(var + '_' + func)
    dfWinFeatures = pd.DataFrame(columns = featureCol) # Temporary DF row will be appended to this. 
    
    # Use chosen variables and functions to calculate feature statistics over the window. Temporarily store in dfWinFeatures.
    for var in chosenVars:
        for func in calculations:
            evalStatement = ('dfWindow["'+var+'"].'+func)
            dfWinFeatures.at[0,var + '_' + func] = (eval(evalStatement))
   

    
    # ASSIGN WINDOW FEATURES TO FALL FEATURE DICTIONARY
    featureValues = []
    eventID = str(eventdf.eventLabel[impact])
    nextLetter = 'A'
    
    # 'If' statement to append a letter to eventID if already exists in the dictionary.
    while eventID in fallFeaturesDict.keys():
        if len(eventID) == 4:
            eventID = eventID[:-1]
        eventID = eventID + nextLetter
        nextLetter = chr(ord(nextLetter)+1) 
        
    # Assign dictionary key and value pair for the fall    
    for i in range(dfWinFeatures.shape[1]): # Column wise
        for j in range (dfWinFeatures.shape[0]): # Row wise
            featureValues.append(dfWinFeatures.iloc[j,i])
   # Update the dictionary.
    fallFeaturesDict.update({eventID:featureValues})
    
            

In [ ]:
print(dfWinFeatures)
print(fallFeaturesDict)

**Get Pre-Impact Feature Dataframe for Whole Session**

In [13]:
# Assign features per fall from dictionary to dataframe.
fallfeaturesdf = pd.DataFrame.from_dict(fallFeaturesDict)
fallfeaturesdf = fallfeaturesdf.transpose()
fallfeaturesdf.columns = dfWinFeatures.columns

# Append a final column based on whether the event IS a right/left fall (1) or is not a right/left fall (0)
rlFall = ['103','108','111','115','117','119','104','109','112','116','118','120']
outputLabel = []
for index in fallfeaturesdf.index:
    if index in rlFall:
        outputLabel.append(1)
    else:
        outputLabel.append(0)
        
fallfeaturesdf['Output'] = outputLabel
fallfeaturesdf

,bAx_mean(),bAx_max(),bAx_min(),bAx_std(),bAy_mean(),bAy_max(),bAy_min(),bAy_std(),bAz_mean(),bAz_max(),...,lAx_std(),lAy_mean(),lAy_max(),lAy_min(),lAy_std(),lAz_mean(),lAz_max(),lAz_min(),lAz_std(),Output
10,0.436966,0.468649,0.420970,0.009372,0.425516,0.448805,0.404430,0.007401,0.608457,0.620313,...,0.009028,0.492980,0.531909,0.464360,0.011156,0.505436,0.518105,0.499443,0.003729,0
10A,0.436736,0.467794,0.417709,0.010127,0.425187,0.441339,0.410204,0.005999,0.609087,0.620683,...,0.009798,0.492852,0.523752,0.472749,0.008972,0.504903,0.514305,0.498177,0.003093,0
117,0.433925,0.537469,0.393352,0.022201,0.417793,0.481732,0.362120,0.017606,0.612248,0.667025,...,0.019391,0.490563,0.534968,0.449043,0.013382,0.499075,0.513619,0.455375,0.009366,1
117A,0.433431,0.539785,0.404867,0.023450,0.414507,0.465779,0.350597,0.020902,0.611077,0.669175,...,0.019182,0.492541,0.552718,0.448973,0.015064,0.498326,0.511208,0.432364,0.012547,0
117B,0.429728,0.489070,0.417934,0.007484,0.418331,0.434744,0.353509,0.012296,0.609363,0.623074,...,0.006765,0.492149,0.533902,0.457617,0.010152,0.499828,0.508019,0.463127,0.008745,0
118,0.432174,0.514371,0.395106,0.017871,0.425469,0.460553,0.389000,0.010337,0.614843,0.691086,...,0.014009,0.488404,0.532094,0.397414,0.019104,0.504019,0.535943,0.490745,0.006596,1
118A,0.431199,0.514776,0.387572,0.016632,0.425244,0.456720,0.400199,0.008112,0.613147,0.701485,...,0.012985,0.490943,0.537239,0.400102,0.017077,0.500962,0.528771,0.484764,0.005988,0
118B,0.432229,0.518060,0.418226,0.014294,0.425264,0.444350,0.404430,0.008513,0.614567,0.688416,...,0.011812,0.487911,0.514182,0.436993,0.012302,0.501065,0.516854,0.494224,0.004155,0
119,0.429561,0.484392,0.398772,0.012119,0.417434,0.433400,0.361648,0.014150,0.613507,0.640332,...,0.009221,0.491491,0.512235,0.466956,0.007295,0.494019,0.504479,0.443809,0.010122,1
119A,0.426642,0.456054,0.402168,0.009674,0.410667,0.432130,0.345296,0.013915,0.619689,0.642668,...,0.009639,0.490828,0.527135,0.457988,0.010185,0.493896,0.503975,0.448661,0.009401,0


**Create Separate DataFrame of Pre-Impact Features based on Fall Type**

In [22]:
allFall_preFeatures = []
sideFall_preFeatures = []
NF_preFeatures = []
fbFall_preFeatures = []

for i in range((fallfeaturesdf.shape[0])):
    if eventdf.actualFallType[i]==2:
        sideFall_preFeatures.append(list(fallfeaturesdf.iloc[i,:]))
        allFall_preFeatures.append(list(fallfeaturesdf.iloc[i,:]))
    elif eventdf.actualFallType[i]==1:
        fbFall_preFeatures.append(list(fallfeaturesdf.iloc[i,:]))
        allFall_preFeatures.append(list(fallfeaturesdf.iloc[i,:]))
    else:
        NF_preFeatures.append(list(fallfeaturesdf.iloc[i,:]))

# Turn all lists into data frame with same column headings. 
allFall_preFeatDF = pd.DataFrame(allFall_preFeatures, columns = fallfeaturesdf.columns) 
sideFall_preFeatDF = pd.DataFrame(sideFall_preFeatures, columns = fallfeaturesdf.columns)
NF_preFeatDF = pd.DataFrame(NF_preFeatures, columns = fallfeaturesdf.columns)
fbFall_preFeatDF = pd.DataFrame(fbFall_preFeatures, columns = fallfeaturesdf.columns)


In [ ]:
# Output the feature dataframe as a document in excel.


# Import machine learning algorithms from sklearn

In [ ]:
# Set parameters.
percent_test = 0.40 # Test size for x data.

# Select the feature data: all rows, all columns except the last (label).
x = fallfeaturesdf.iloc[:,:-1].values
# Select the label column as the output label.
y = fallfeaturesdf['Output']

# Split the data into training and testing data.
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = percent_test, random_state = 0)

#print(x_train)
#print(y_train)

In [27]:
# SVM: Side Fall (1) versus Not Side Fall (0) For Single Subject Session

SVC_model = SVC()
SVC_model.fit(x_train,y_train)
SVC_prediction = SVC_model.predict(x_test)

print(accuracy_score(SVC_prediction, y_test))


0.75


C:\Users\obotonis\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [34]:
# KNN: Side Fall (1) versus Not Side Fall (0) for Single Subject Session

KNN_model = KNeighborsClassifier(n_neighbors = 5)
KNN_model. fit(x_train, y_train)
KNN_prediction = KNN_model.predict(x_test)

print(accuracy_score(KNN_prediction, y_test))

0.7916666666666666


In [ ]:

        # *** This is easier: dfWindow.describe() gives direct statistics on every column.
        df.describe().transpose()
    
        # OR pip install pandas-profiling
        df.profile_report() # Essentials, quantile statistics, descriptive statistics, most frequent values, histogram, correlations, etc. EVERYTHING.
        # This is column wise. We would want some of these to be between sets of data/columns.

    # Assign key:value pair to the fallFeaturesDict
 
    
    # Allow program to re-loop and perform again for another fall.